In this brief workflow, we derive a flood map using the Protogen flood mapper and then convert it into a shapefile using gdal.

Define environment variables and create gbdx interface.

In [ ]:
import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools

gbdx = gbdxtools.Interface()

Create protogen-based flood water mapping task.

In [ ]:
import protogen
import pickle

p = protogen.Interface('crime', 'flood_water_mapper')
p.crime.flood_water_mapper.tolerance = 50               # [%] aim for water bodies only
p.crime.flood_water_mapper.moisture = 3                 # [%] retain all candidates
p.crime.flood_water_mapper.binary_output = True         # output format
p.crime.flood_water_mapper.min_size = 1000              # noise: average housing roof size in square meters 
p.crime.flood_water_mapper.min_width = 10               # noise: road width in meters
p.image_config.bands = [1, 2, 3, 4, 5, 6, 7, 8]
p.verbose = True

fld = gbdx.Task('protogen-runner')
fld.inputs.pickle = pickle.dumps(p)

Define and run the workflow.

In [ ]:
from os.path import join
import uuid

wf_ids = {}

# location of acomped multispectral image
input_location = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/flood-water/north-carolina/10400100238BDE00/ms'

# where to save the shapefile
output_location = join('platform-stories/trial-runs/', str(uuid.uuid4()))

# specify input to flood mapper
fld.inputs.image = input_location

# vectorize output of flood mapper with gdal
vectorize = gbdx.Task('gdal-cli')
vectorize.inputs.data = fld.outputs.output.value
vectorize.inputs.execution_strategy = 'runonce'
vectorize.inputs.command = """gdal_polygonize.py -f "ESRI Shapefile" $indir/*.tif $outdir/vectors.shp"""
## this is for geojson format
#vectorize.inputs.command = """gdal_polygonize.py -f "GeoJSON" $indir/*.tif $outdir/vectors.geojson"""

wf = gbdx.Workflow([fld, vectorize])
wf.savedata(vectorize.outputs.data, join(output_location))
wf.execute()

In [ ]:
print output_location
print wf.status